# Mass Completeness of BGS selection

Two kinds of Mass supersets are considered, one where we consider the SV sample as superset and another where we consider a $z<21$ sample as the superset.

In [ ]:
from pathlib import Path
from importlib import reload
import joblib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as lines
from astropy.table import Table

import mpl_scatter_density
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

from desitarget.sv1.sv1_targetmask import desi_mask, bgs_mask
import utils
reload(utils)

In [ ]:
survey = "south" # choose from `north` or `south

In [ ]:
my_path = Path("/global/cscratch1/sd/bid13/LRG")
bgs_path = my_path / ("BGS_SV_" + survey + ".fits")
mag_lim_path = my_path / ("mag_lim_"+survey+".fits")

In [ ]:
bgs_sv_cat = Table.read(bgs_path) # read in the SV data
good_mass_mask = (bgs_sv_cat["logmass"]>0) & (bgs_sv_cat["logmass"]<12) # remove points with no mass values
bgs_sv_cat = bgs_sv_cat[good_mass_mask]

In [ ]:
bgs_bright_mask = ((bgs_sv_cat["SV1_BGS_TARGET"] & bgs_mask.mask("BGS_BRIGHT_"+survey.upper()))>0)
bgs_faint_mask = ((bgs_sv_cat["SV1_BGS_TARGET"] & bgs_mask.mask("BGS_FAINT_"+survey.upper()))>0)
bgs_faint_ext_mask = ((bgs_sv_cat["SV1_BGS_TARGET"] & bgs_mask.mask("BGS_FAINT_EXT_"+survey.upper()))>0)
total = bgs_bright_mask | bgs_faint_mask | bgs_faint_ext_mask

### Mass completeness with spectroscopic redshifts

In [ ]:
z_cat = Table.read("/global/cfs/cdirs/desi/survey/catalogs/SV1/redshift_comps/cascades/3/BGS_ANY/alltiles_BGS_ANYzinfo.fits")
z_cat = z_cat[(z_cat["FIBERSTATUS"]==0) & (z_cat["ZWARN"]==0)]
z_cat = z_cat[z_cat["TARGETS"] == "BGS+MWS"]
z_cat = z_cat[z_cat["PHOTSYS"]==survey.upper()[0]]
z_cat = z_cat[z_cat["SPECTYPE"]=="GALAXY"]

names = [name for name in z_cat.colnames if len(z_cat[name].shape) <= 1]
z_cat = z_cat[names].to_pandas()
# z_cat = z_cat[z_cat["TILEID"].isin([80605,80607,80609,80620,80622, 68000, 68001, 68002])]
z_cat = z_cat[z_cat.duplicated(subset="TARGETID")][["TARGETID", "Z"]]

In [ ]:
z_cat = z_cat.merge(bgs_sv_cat.to_pandas(), how="inner", on= "TARGETID")

In [ ]:
# Get masses based on spec-z instead of photoz
regrf = joblib.load(f'/global/cfs/cdirs/desi/users/rongpu/ls_dr9.0_photoz/stellar_mass/rf_20210206_{survey}-mgc_redshift.pkl')
data = np.column_stack((z_cat['gmag']-z_cat['rmag'], z_cat['rmag']-z_cat['zmag'], z_cat['zmag']-z_cat['w1mag'], z_cat['w1mag']-z_cat['w2mag'], z_cat['Z']))
distmod = cosmo.distmod(z_cat['Z'])
m_to_l_predict = regrf.predict(data)
z_cat["logmass_z"] = m_to_l_predict + 0.4*(4.45-(z_cat['zmag'] - distmod.to_value() ))

In [ ]:
bgs_bright_mask = ((z_cat["SV1_BGS_TARGET"] & bgs_mask.mask("BGS_BRIGHT_"+survey.upper()))>0)
bgs_faint_mask = ((z_cat["SV1_BGS_TARGET"] & bgs_mask.mask("BGS_FAINT_"+survey.upper()))>0)
bgs_faint_ext_mask = ((z_cat["SV1_BGS_TARGET"] & bgs_mask.mask("BGS_FAINT_EXT_"+survey.upper()))>0)
bgs_mask = bgs_bright_mask | bgs_faint_mask | bgs_faint_ext_mask

bgs_bright = z_cat[bgs_bright_mask]
bgs_faint = z_cat[bgs_faint_mask]
bgs = z_cat[bgs_mask]

In [ ]:
z_min=0. #Minimum of the redshift range
z_max=0.6#Maximum of the redshift range
z_bins=8 #Number of redhshift bins to make
mass_bins=50 #Number of mass bins to make

z_edges=np.linspace(z_min,z_max,z_bins+1)

In [ ]:
fig, axs = plt.subplots(2,4, figsize=(15, 8), facecolor='w', edgecolor='k', sharex="col", sharey='row')
axs = axs.ravel()

for i in range(z_bins):
    #select objects in the redshift bin
    mass_z_bright = bgs_bright[ (bgs_bright["Z"]>=z_edges[i]) & (bgs_bright["Z"]<z_edges[i+1])] ["logmass_z"]    
    mass_z_faint = bgs_faint[ (bgs_faint["Z"]>=z_edges[i]) & (bgs_faint["Z"]<z_edges[i+1])] ["logmass_z"]
    
    mass_bgs_z = bgs[ (bgs["Z"]>=z_edges[i]) & (bgs["Z"]<z_edges[i+1])] ["logmass_z"]
    
    #generate mass points
    mass_ticks = np.linspace( mass_bgs_z.min() ,mass_bgs_z.max(),mass_bins)
    completeness_bright=[]
    mass_plot_ticks_bright=[]
    completeness_faint=[]
    mass_plot_ticks_faint=[]
    
    #Find fraction of objects greater than the given mass
    for j in range(mass_bins):
        #bright
        if ((mass_z_bright>=mass_ticks[j]).sum())>0: # Points will be plotted only if there are more than 50 data points in a bin
            completeness_bright.append( (mass_z_bright>=mass_ticks[j]).sum()/(mass_bgs_z>=mass_ticks[j]).sum() )
            mass_plot_ticks_bright.append(mass_ticks[j])
        #faint
        if ((mass_z_faint>=mass_ticks[j]).sum())>0: # Points will be plotted only if there are more than 50 data points in a bin
            completeness_faint.append( (mass_z_faint>=mass_ticks[j]).sum()/(mass_bgs_z>=mass_ticks[j]).sum() )
            mass_plot_ticks_faint.append(mass_ticks[j])
    
    label = str(np.round(z_edges[i],3))+r"$\leq z <$"+str(np.round(z_edges[i+1],3))
#     df = pd.DataFrame({"logmass": mass_plot_ticks_bright, "completeness": completeness_bright})
#     df.to_csv("BGS_"+str(np.round(z_edges[i],3))+"-"+str(np.round(z_edges[i+1],3))+"_"+survey+".csv")
    tot_count, mass_bin_edges = np.histogram(mass_bgs_z, bins=mass_bins)
    sel_count_bright, mass_bin_edges = np.histogram(mass_z_bright, bins=mass_bin_edges)
    sel_count_faint, mass_bin_edges = np.histogram(mass_z_faint, bins=mass_bin_edges)
    
    axs[i].plot(mass_plot_ticks_bright,completeness_bright, ls="--", color="k")
#     axs[i].plot(mass_plot_ticks_faint,completeness_faint, ls="--", color="red")
    utils.hist_on_binned_array(tot_count/tot_count.max(), mass_bin_edges, ax=axs[i], alpha=0.3, color="C0")
    utils.hist_on_binned_array(sel_count_bright/tot_count.max(), mass_bin_edges, ax=axs[i], histtype="step", color="k")
#     utils.hist_on_binned_array(sel_count_faint/tot_count.max(), mass_bin_edges, ax=axs[i], histtype="step", color="red")
    axs[i].set_title(label)
    axs[i].set_ylim(0,1.1)
    axs[i].set_xlim(8,12)
    axs[i].grid(alpha=0.5)
fig.text(0.5, 0.04, r"Stellar Mass ($\log(M_{\odot})$)", ha='center',size=20) #Common x label
fig.text(0.06, 0.5, "Fraction Selected", va='center', rotation='vertical',size=20) #Common y label


handle1 = lines.Line2D([], [], c='k', ls ="--", lw=2)
handle2 = lines.Line2D([], [], c='red', ls ="--", lw=2)
fig.legend( (handle1,), ("BGS Bright", ), loc="center right", bbox_to_anchor=(0.5,1))